In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 40)

## Read csv files from db directory

In [5]:
# file and DB URL
db_url = "C:/Users/hyo05/Documents/Portfolio/Apple transaction/csv/"
file_url = db_url + "Apple Card Transactions - April 2021.csv"

# 1.2 Transaction datre
Step 2. 

# Step 1. Data cleaning

## 1.1 Reading a csv file and create data frame

### Rematch falsely matched pair of Merchant and Category.  

In [6]:
# Define mecrchant and category dictionary for pair of merchant and category that does not match up with each other in the original data set.
apple_category_dict = {
    'Apple Services' : 'Entertainment',
    'Extra Space Storage' : 'Utilities',
    'Geico' : 'Automobile',
    'City of LA Dot Pvb Pbp' : 'Automobile',
    'Soundcloud' : 'Subscriptions',
    'T-Mobile' : 'Utilities',
    'FasTrak' : 'Automobile',
    'Marshalls' : 'Shopping',
    'trashwarrior.com' : 'Utilities',
    'Koreana Plaza Market' : 'Grocery',
    'Airbnb' : 'Travel',
    'Daily Cash Adjustment' : 'Other',
    'Chase Jayne' : 'Unknown',
    'Lim*Ride Cost' : 'General',
    "Zoomy's Santa Nella" : "Unknown",
    'H Mart' : 'Grocery',
    'Soundcloud' : 'Subscriptions',
    'Chevron' : 'Automobile',
    'Homeowners Insurance' : 'Utilities',
    'Tokyo Fish Market' : 'Grocery',
    'Safeway' : 'Grocery',
    'Discount Cigarette' : 'Entertainment',
    'Amazon Marketplace' : 'Grocery',
    'Amazon Music' : 'Subscription',
    'Dollar Tree' : 'Shopping',
    'www.coursehero.com' : 'Subscription',
    'updater.com' : 'Unknown',
    'Stellar' : 'General',
    'Amazon Web Services' : 'General',
    'The One Wireless & Gif' : 'Entertainment',
    'Amazon' : 'Shopping',
    'Big AL S' : 'Entertainment',
    'The UPS Store' : 'General',
    'Hornblower Yachts' : 'Unknown',
    'Card payment' : 'Payment',
    'Home Depot' : 'Shopping',
    'Aventri Event Reg' : 'Unknown',
    'UC Awpe Fee' : 'Tuition',
    'Element Vape' : 'Entertainment',
    'CVS Pharmacy' : 'General',
    'Amazon Prime' : 'Subscriptions',
    'FedEx' : 'General',
    'Service Dog Certificat' : 'General',
    'Contra Costa Comm Coll' : 'Tuition'
    }

In [7]:
apple_merchant_long_dict = {
    "Ach Deposit Internet Transfer From Account Ending In 3437" : "Card payment",
    'Geico *auto One Geico Plaza 800-841-3000 20076 Dc Usa (dispute Credit)' : 'Geico dispute credit',
    'Postmates Membership 201 3rd Stsuite 200 8778877815 94103 Ca Usa (dispute Credit)' : 'Postmate dispute credit',
    'Aventri Event Reg 7700 Eastport Parkway 2034039470 06854 Ct Usa (dispute Credit)' : 'Aventri dispute credit',
    'Amazon Web Services 440 Terry Ave N Aws.amazon.co98109 Wa Usa (dispute Credit)' : 'AWS dispute credit'
}

In [8]:
# Search for merchants in the dictionary and edit category if the merchant exists. 
def match_category(df, long_dict, category_dict):
    
    for index, row in df.iterrows():
        # Shortening long mervhant names.
        if row['Merchant'] in long_dict.keys():
            df.at[index, 'Merchant'] = long_dict.get(row['Merchant'])

        # Matching correct category.
        if row['Merchant'] in category_dict.keys():
            df.at[index, 'Category'] = category_dict.get(row['Merchant'])

    return df

### Divide 'Transaction Date' column into 3 columns: Month, Day, Year

In [9]:
# Cleaning date into Month, Day, Year for future operations
def clean_date(df):

    month = np.array([]).astype(int)
    day = np.array([]).astype(int)
    year = np.array([]).astype(int)

    for index, row in df.iterrows():
        date = row['Transaction Date'].split('/')
        month = np.append(month, date[0])
        day = np.append(day, date[1])
        year = np.append(year, date[2])

    # Insert Month, Day, Year column and drop 'Transaction Date' column.
    df.insert(0, 'Month', month)
    df.insert(1, 'Day', day)
    df.insert(2, 'Year', year)
    df = df.drop(columns=['Transaction Date'])

    return df

In [10]:
def read_csv(f_url, long_dict, cat_dcit):
    # Reading csv file for selected columns and create and sort dataframe by the Transaction date
    f_df = pd.read_csv(f_url)[['Transaction Date', 'Merchant', 'Category', 'Type', 'Amount (USD)']]
    f_df = f_df.sort_values(by=['Transaction Date'])

    # Cleaning Merchant and Category
    f_df = match_category(f_df, long_dict, cat_dcit)

    # Clean 'Transaction Date' column. 
    f_df = clean_date(f_df)

    return f_df

april_df = read_csv(file_url, apple_merchant_long_dict, apple_category_dict)
print(april_df)

   Month Day  Year        Merchant       Category      Type  Amount (USD)
40    03  31  2021        T-Mobile      Utilities  Purchase         26.68
39    04  01  2021           Geico     Automobile  Purchase        264.66
38    04  02  2021      Soundcloud  Subscriptions  Purchase          4.99
36    04  03  2021  Apple Services  Entertainment  Purchase          3.99
37    04  04  2021    Card payment        Payment   Payment       -343.30
..   ...  ..   ...             ...            ...       ...           ...
4     04  29  2021     Papa John's    Restaurants  Purchase         21.84
3     04  29  2021    Seoul Hotdog    Restaurants  Purchase         10.90
2     04  29  2021  Apple Services  Entertainment  Purchase         19.99
1     04  29  2021  Apple Services  Entertainment  Purchase          4.99
0     04  30  2021  Apple Services  Entertainment  Purchase          2.99

[41 rows x 7 columns]


## 1.2 Read all csv files from DB.

### Read all files from DB.

In [11]:
def initiate_db():
    # Create empty dataframe with cloumn labels 
    c_label = ['Month', 'Day', 'Year', 'Merchant', 'Category', 'Type', 'Amount (USD)']
    raw_df = pd.DataFrame(columns=c_label)

    # For each files in the DB append raw 
    for i in os.listdir(db_url):
        f_url = db_url + i
        tmp = read_csv(f_url, apple_merchant_long_dict, apple_category_dict)
        raw_df = raw_df.append(tmp, ignore_index=True)

    # Sorting by date
    raw_df = raw_df.sort_values(by=['Year', 'Month', 'Day'])

    # Reindexing
    raw_df = raw_df.reset_index().drop(columns='index')

    return raw_df

db = initiate_db()
db

,Month,Day,Year,Merchant,Category,Type,Amount (USD)
0,05,16,2020,Postmates,Restaurants,Purchase,22.93
1,05,19,2020,In-N-Out Burger,Restaurants,Purchase,16.30
2,05,19,2020,Lim*Ride Cost,General,Purchase,2.99
3,05,21,2020,Jamba Juice,Restaurants,Purchase,13.78
4,05,21,2020,CVS Pharmacy,General,Purchase,54.61
...,...,...,...,...,...,...,...
303,04,29,2021,Papa John's,Restaurants,Purchase,21.84
304,04,29,2021,Seoul Hotdog,Restaurants,Purchase,10.90
305,04,29,2021,Apple Services,Entertainment,Purchase,19.99
306,04,29,2021,Apple Services,Entertainment,Purchase,4.99


## Update dictionary

### Create two column dataframe of Merchant and Category for each unique merchant. 

In [12]:
# Create a dataframe of unique Merchant and Category pair.
def create_merchant_category_df(df): 
    category_arr = np.array([])
    merchant_ls = list(set(df.Merchant))

    # Append category_arr for each merchant in the Mercnat set.
    for i in merchant_ls:
        category = df.loc[df['Merchant'] == i]['Category'].values[0]
        category_arr = np.append(category_arr, category)

    # Create data frame of unique merchant and category pair.
    merchant_category_df = pd.DataFrame({
        "Merchant": merchant_ls,
        "Category": category_arr
    })

    return merchant_category_df

merchant_category_df = create_merchant_category_df(db)
merchant_category_df

,Merchant,Category
0,Apple Services,Entertainment
1,Humphry Slocombe I,Restaurants
2,The UPS Store,General
3,Marshalls,Shopping
4,Homeowners Insurance,Utilities
...,...,...
58,UC Awpe Fee,Tuition
59,Daily Cash Adjustment,Other
60,Papa John's,Restaurants
61,Amazon Music,Subscription


### Create two column dataframe that shows merchants and their catergory that are not tracked in the apple_category_dictionary.

#### Ignore "Restaurants" category as they matches with their merchants.

In [13]:
# Iterate through dataframe and search for merchants not listed in the merchant and category dictionary.
def untracked_merchant(df):

    # Initialize empty arrays.
    category_arr = np.array([])
    merchant_arr = np.array([])
    merchant_ls = list(set(df.Merchant))
    
    # Iterate list of unique merchants and search for merchants not listed in the dictionary.
    for i in merchant_ls:    
        if i not in apple_category_dict.keys():
            category = df.loc[df['Merchant']==i]['Category'].values[0]
            # Ignore category "Restaurant"
            if category == "Restaurants":
                continue

            merchant_arr = np.append(merchant_arr, i)
            category_arr = np.append(category_arr, category)

    
    # Create data frame of unique merchant and category pair.
    untracked_merchant_df = pd.DataFrame({
        "Merchant": merchant_arr,
        "Category": category_arr
    })

    return untracked_merchant_df

untracked_merchant(db)

,Merchant,Category
0,AWS dispute credit,Credit
1,Geico dispute credit,Credit
2,Postmate dispute credit,Credit
3,Aventri dispute credit,Credit


#### 'Merchants' who has'Category' value w/ 'Other' needs to be updated
#### 'Unknown' used for transactions cannot be remembered.  

In [17]:
# By each category
for category in list(set(db.Category)):
    print(category)
    tmp = db.loc[db['Category'] == category]
    tmp = tmp.groupby(by=["Merchant",'Year', 'Month']).sum()
    print(tmp)

Utilities
                                 Amount (USD)
Merchant             Year Month              
Extra Space Storage  2020 05            79.20
                          06            55.20
                          07            55.20
                          08            55.20
                          09            55.20
                          10            55.20
                          11            55.20
                          12            55.20
                     2021 01            48.00
                          02            24.00
                          03           102.67
                          04            59.20
Homeowners Insurance 2020 11           160.00
T-Mobile             2020 12            37.63
                     2021 03            26.68
trashwarrior.com     2021 01            95.00
Subscriptions
                         Amount (USD)
Merchant     Year Month              
Amazon Prime 2021 01            64.61
Soundcloud   2021 01             4

In [15]:
# By each category and month
print(db.groupby(by=['Category', 'Year', 'Month']).sum())

                       Amount (USD)
Category   Year Month              
Automobile 2020 05             2.09
                08           290.95
                09           460.40
                11           757.72
                12           465.38
...                             ...
Utilities  2020 12            92.83
           2021 01           143.00
                02            24.00
                03           129.35
                04            59.20

[102 rows x 1 columns]


## Remove all files after reading them
## Check whether any file in the directory for update
## 

# 2. Analysis
## 2.1 By type
### purchase and payment
## 2.2 Merchant and Disctionary
### Unique merchant
#### Update merchant dictionary
### Monthly spending on each merchant and category 
## 2.3 Monthly report 

In [16]:
purchase = april_df[april_df['Type'] == "Purchase"]
print(purchase)
payment = april_df[april_df['Type'] == "Payment"]
print(payment)

   Month Day  Year                Merchant       Category      Type  \
40    03  31  2021                T-Mobile      Utilities  Purchase   
39    04  01  2021                   Geico     Automobile  Purchase   
38    04  02  2021              Soundcloud  Subscriptions  Purchase   
36    04  03  2021          Apple Services  Entertainment  Purchase   
35    04  08  2021          Apple Services  Entertainment  Purchase   
34    04  08  2021          Apple Services  Entertainment  Purchase   
33    04  13  2021          Apple Services  Entertainment  Purchase   
31    04  18  2021     Extra Space Storage      Utilities  Purchase   
29    04  19  2021                 Safeway        Grocery  Purchase   
28    04  20  2021          Apple Services  Entertainment  Purchase   
30    04  20  2021          Apple Services  Entertainment  Purchase   
26    04  21  2021          Apple Services  Entertainment  Purchase   
25    04  21  2021          Apple Services  Entertainment  Purchase   
24    